# Chat with database

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]

llm = ChatOpenAI(openai_api_key=openai_api_key, model = "gpt-4o-mini", temperature=0)

# Create database




In [2]:
import os
import requests
import sqlite3

# Define the path to the Chinook SQLite database
project_path = os.getcwd()  # Gets the current working directory
db_path = os.path.join(project_path, "Chinook.db")

# Download the Chinook SQL script
url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
response = requests.get(url)
sql_script = response.text

# Create and populate the SQLite database
connection = sqlite3.connect(db_path)
connection.executescript(sql_script)
connection.close()

# Verify the database connection and list tables
connection = sqlite3.connect(db_path)
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)
connection.close()


Tables in the database: [('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///"+db_path)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [4]:
db.run("SELECT * FROM Artist LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# Asking database for data

In [5]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'```sql\nSELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee";\n```'

In [ ]:
db.run(response) # this will throw an error because of additional characters in the query

OperationalError: (sqlite3.OperationalError) near "```sql
SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee";
```": syntax error
[SQL: ```sql
SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee";
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [7]:
new_llm = ChatOpenAI(openai_api_key=openai_api_key, model = "gpt-4o", temperature=0)
new_chain = create_sql_query_chain(new_llm, db)
new_response = chain.invoke({"question": "How many employees are there"})

In [8]:
db.run(new_response) # Again error having better model

OperationalError: (sqlite3.OperationalError) near "```sql
SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee";
```": syntax error
[SQL: ```sql
SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee";
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [9]:
new_response = chain.invoke({"question": "How many employees are there? Use sqlite syntax"})

In [10]:
db.run(new_response) # Still nothing

OperationalError: (sqlite3.OperationalError) near "SQLQuery": syntax error
[SQL: SQLQuery: SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee";]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [11]:
# Let's review it
print(new_response)

SQLQuery: SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee";


In [12]:
# Try simple fix
response = response.replace("SQLQuery: ", "").replace("```", "").replace("\n", "").replace("sql", "")
response

'SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee";'

In [13]:
db.run(response)

'[(8,)]'

As we can see - it's not that simple without additional mechanisms. It may require graphs.
See more here: https://langchain-ai.github.io/langgraph/tutorials/sql-agent/#define-tools-for-the-agent